<a href="https://colab.research.google.com/github/sarahatami/Pytorch_Segmentation/blob/master/Supervised_Pytorch_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os
import numpy as np
from os import listdir
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch
import torchvision.models.segmentation as seg
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import sys
from collections import namedtuple
from PIL import Image

In [ ]:
# city escape pictures
! wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=sarahatami&password=...&submit=Login' https://www.cityscapes-dataset.com/login/
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1 #gt
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3 #image

In [ ]:
!unzip /content/gtFine_trainvaltest.zip
!rm /content/README
!rm /content/index.html
!rm /content/license.txt
!unzip /content/leftImg8bit_trainvaltest.zip
!rm /content/gtFine_trainvaltest.zip
!rm /content/leftImg8bit_trainvaltest.zip

In [ ]:
Label = namedtuple( 'Label' , ['name', 'id', 'trainId', 'category', 'categoryId', 'hasInstances', 'ignoreInEval', 'color',  ] )
labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
  # Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
    Label(  'car'                  , 26 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

In [ ]:
np.set_printoptions(threshold = np.inf)
transform_ann = transforms.Compose([transforms.Resize((205, 410))])

def gt_to_label(pil_image):
  transformed_ann = transform_ann(pil_image)
  arr_image = np.array(transformed_ann)
  label_map = np.zeros(arr_image.shape[0:2], dtype=np.uint8) # (205, 410)
  for label in labels:
    i = np.all(arr_image == label.color, axis=-1)
    label_map[i] = label.id # (205, 410)
  return label_map

In [ ]:
### Defining the Dataset class ###
class Cityscapes(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.images_dir = '/content/leftImg8bit/train/bremen'
        self.images = sorted(os.listdir(self.images_dir))
        self.gt_dir = '/content/gtFine/train/bremen'
        self.gts = sorted([f for f in os.listdir(self.gt_dir) if f.endswith("color.png")])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        gt_path = os.path.join(self.gt_dir, self.gts[idx])

        pil_image = Image.open(img_path).convert("RGB")  #<class 'PIL.Image.Image'>
        pil_gt = Image.open(gt_path).convert("RGB") #<class 'PIL.Image.Image'>
        gt = gt_to_label(pil_gt)
        if self.transform:
            img = self.transform(pil_image)
        return img, gt

transform_img = transforms.Compose([transforms.Resize((205, 410)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

trainset = Cityscapes(transform=transform_img)
dataloader_train = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=False)

In [ ]:
learning_rate = 1e-6
criterion = CrossEntropyLoss()
num_epochs = 100

In [ ]:
color_map = [(  0,  0,  0) ,
             (  0,  0,  0) ,
             (  0,  0,  0) ,
             (  0,  0,  0) ,
             (  0,  0,  0) ,
             (111, 74,  0) ,
             ( 81,  0, 81) ,
             (128, 64,128) ,
             (244, 35,232) ,
             (250,170,160) ,
             (230,150,140) ,
             ( 70, 70, 70) ,
             (102,102,156) ,
             (190,153,153) ,
             (180,165,180) ,
             (150,100,100) ,
             (150,120, 90) ,
             (153,153,153) ,
             (153,153,153) ,
             (250,170, 30) ,
             (220,220,  0) ,
             (107,142, 35) ,
             (152,251,152) ,
             ( 70,130,180) ,
             (220, 20, 60) ,
             (255,  0,  0) ,
             (  0,  0,142) ,
             (  0,  0, 70) ,
             (  0, 60,100) ,
             (  0,  0, 90) ,
             (  0,  0,110) ,
             (  0, 80,100) ,
             (  0,  0,230) ,
             (119, 11, 32) ,
             (  0,  0,142) ]

#################################################################

****DeepLabV3 model with a ResNet-50 backbone****

In [ ]:
num_classes=34
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = seg.deeplabv3_resnet50(pretrained=True)
# model = seg.lraspp_mobilenet_v3_large( pretrained=True)
# model = seg.deeplabv3_mobilenet_v3_large(pretrained=True)

last_layer = model.classifier[-1]
model.classifier[4]=torch.nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))
model = model.to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader_train):
        inputs, annotations = data # [bs, 3, 205, 410], [bs, 205, 410]
        inputs = inputs.to(device)
        annotations = annotations.to(device).long()
        # inputs=torch.autograd.Variable(inputs,requires_grad=False)

        optimizer.zero_grad()
        model.train()
        outputs = model(inputs)['out']

        loss = criterion(outputs, annotations)
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f"model is at epoch {epoch}, step {i}")
torch.save(model.state_dict(), f"{epoch}.torch")

In [ ]:
# test
model = seg.deeplabv3_resnet50(pretrained=True)
model.classifier[4]=torch.nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))

model_path="/content/99.torch"
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

for i, data in enumerate(dataloader_train):
  images, annotations = data
  images = images.to(device)
  outputs = model(images)['out'] #[4, 34, 205, 410]

  segm = torch.argmax(outputs, 1).cpu().detach().numpy() #(4, 205, 410)
  labelmap=segm[0,:,:]
  rgbimage = np.zeros((205, 410, 3), dtype=np.uint8)

  for i in range(labelmap.shape[0]):
    for j in range(labelmap.shape[1]):
        color_index = labelmap[i, j]
        rgbimage[i, j] = color_map[color_index]  # Assign the corresponding RGB color

  out_image = Image.fromarray(rgbimage)
  plt.imshow(out_image)
  plt.show()
  # stop

######################################################################

****Lite R-ASPP Network model with a MobileNetV3-Large backbone****

In [ ]:
num_classes=34
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model = seg.deeplabv3_resnet50(pretrained=True)
model = seg.lraspp_mobilenet_v3_large( pretrained=True)
# model = seg.deeplabv3_mobilenet_v3_large(pretrained=True)

model.classifier.high_classifier = torch.nn.Conv2d(model.classifier.high_classifier.in_channels
                                                   , num_classes, kernel_size=(1, 1), stride=(1, 1))
model.classifier.low_classifier = torch.nn.Conv2d(model.classifier.low_classifier.in_channels
                                                   , num_classes, kernel_size=(1, 1), stride=(1, 1))
model = model.to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader_train):
        inputs, annotations = data # [bs, 3, 205, 410], [bs, 205, 410]
        inputs = inputs.to(device)
        annotations = annotations.to(device).long()
        # inputs=torch.autograd.Variable(inputs,requires_grad=False)

        optimizer.zero_grad()
        model.train()
        outputs = model(inputs)['out']

        loss = criterion(outputs, annotations)
        loss.backward()
        optimizer.step()

    if i % 100 == 0:
        print(f"model at epoch {epoch}, step {i}")
torch.save(model.state_dict(), f"{epoch}.torch")

In [ ]:
# test
model = seg.deeplabv3_resnet50(pretrained=True)
model.classifier[4]=torch.nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))

model_path="/content/99.torch"
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

for i, data in enumerate(dataloader_train):
  images, annotations = data
  images = images.to(device)
  outputs = model(images)['out'] #[4, 34, 205, 410]

  segm = torch.argmax(outputs, 1).cpu().detach().numpy() #(4, 205, 410)
  labelmap=segm[0,:,:]
  rgbimage = np.zeros((205, 410, 3), dtype=np.uint8)

  for i in range(labelmap.shape[0]):
    for j in range(labelmap.shape[1]):
        color_index = labelmap[i, j]
        rgbimage[i, j] = color_map[color_index]  # Assign the corresponding RGB color

  out_image = Image.fromarray(rgbimage)
  plt.imshow(out_image)
  plt.show()
  # stop

##########################################################################

 ****DeepLabV3 model with a MobileNetV3-Large backbone****

In [ ]:
num_classes=34
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model = seg.deeplabv3_resnet50(pretrained=True)
# model = seg.lraspp_mobilenet_v3_large( pretrained=True)
model = seg.deeplabv3_mobilenet_v3_large(pretrained=True)

model.classifier[4]=torch.nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))
last_layer = model.classifier[-1]
model = model.to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    print(epoch)
    for i, data in enumerate(dataloader_train):
        inputs, annotations = data # [bs, 3, 205, 410], [bs, 205, 410]
        inputs = inputs.to(device)
        annotations = annotations.to(device).long()
        # inputs=torch.autograd.Variable(inputs,requires_grad=False)

        optimizer.zero_grad()
        model.train()
        outputs = model(inputs)['out']

        loss = criterion(outputs, annotations)
        loss.backward()
        optimizer.step()

    if i % 100 == 0:
      print(f"model is at epoch {epoch}, step {i}")
torch.save(model.state_dict(), f"{epoch}_2nd.torch")

In [ ]:
# test
model = seg.deeplabv3_resnet50(pretrained=True)
model.classifier[4]=torch.nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))

model_path="/content/99.torch"
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

for i, data in enumerate(dataloader_train):
  images, annotations = data
  images = images.to(device)
  outputs = model(images)['out'] #[4, 34, 205, 410]

  segm = torch.argmax(outputs, 1).cpu().detach().numpy() #(4, 205, 410)
  labelmap=segm[0,:,:]
  rgbimage = np.zeros((205, 410, 3), dtype=np.uint8)

  for i in range(labelmap.shape[0]):
    for j in range(labelmap.shape[1]):
        color_index = labelmap[i, j]
        rgbimage[i, j] = color_map[color_index]  # Assign the corresponding RGB color

  out_image = Image.fromarray(rgbimage)
  plt.imshow(out_image)
  plt.show()
  # stop